# HyperPipes

HyperPipes é um algoritmo de classificação desenvolvido por Lucio de Souza Coelho,  Universidade Federal de Minas Gerais, e posteriormente aperfeiçoado por Len Trigg, The University of Waikato, originalmente publicado na versão estável 3.2 do Weka em 1999. O algoritmo básico consiste em criar hipercubos definidos a partir das fronteiras de cada dimensão das classes presentes, em novas instâncias são classificadas de acordo com o hipercubo que mais engloba-la. Apesar de ser um algoritmo muito simples, possui como maior vantagem ser extremamente rápido, devido a sua baixa complexidade de tempo e memória computacional. Ele funciona bem quando há uma quantidade muito grande de atributos e/ou as classes são bem separadas e definidas no espaço amostral.

## Implementação

O algoritmo é implementado abaixo. Separou-se o conceito de um único *HiperPipe* (representando uma classe), do conceito de *n* multiplos *HiperPipes*, para facilitar a compreensão da implementação.

In [1]:
import numpy as np

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, load_digits
from sklearn.cross_validation import StratifiedKFold, cross_val_score

C:\Users\altargin\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class HyperPipe:
    
    def __init__(self):
        self.n_dimensions = 0
        self.numerical_bounds = []

    def fit(self, data_x, target_class):
        self.target_class = target_class
        self.n_dimensions = data_x.shape[1]

        # Initializes bounds
        for i in range(self.n_dimensions):
            bounds = []
            bounds.append(float('+inf'))  # lower bound
            bounds.append(float('-inf'))  # upper bound
            self.numerical_bounds.append(bounds)

        # Add instances
        for i in range(data_x.shape[0]):
            self.__add_instance__(data_x[i])

        return None

    def __add_instance__(self, data_x):
        #check boundaries
        for i in range(self.n_dimensions):
            if(data_x[i] < self.numerical_bounds[i][0]):
                self.numerical_bounds[i][0] = data_x[i]
            if(data_x[i] > self.numerical_bounds[i][1]):
                self.numerical_bounds[i][1] = data_x[i]

        return None

    def partial_contains(self, data_x):
        count = 0
        for i in range(self.n_dimensions):
            if(data_x[i] > self.numerical_bounds[i][0] and data_x[i] < self.numerical_bounds[i][1]):
                count += 1
        score = float(count) / self.n_dimensions

        return (score, self.target_class)

In [3]:
from sklearn.base import BaseEstimator

class HyperPipes(BaseEstimator):

    def __init__(self):
        self.hyper_pipes = []

    def fit(self, data_x, data_y):
        self.y_unique_values, self.y_unique_indices = np.unique(
            data_y, return_inverse=True)
        self.n_y_unique = self.y_unique_values.shape[0]
        self.hyper_pipes = [HyperPipe() for i in range(self.n_y_unique)]

        for i in range(self.n_y_unique):
            target_class = self.y_unique_values[i]
            target_class_indices = np.where(data_y == target_class)
            data_x_filtered = data_x[target_class_indices]
            self.hyper_pipes[i].fit(data_x_filtered, target_class)

        return self

    def predict(self, data_x):
        predictions = []
        for instance in data_x:
            partial_results = []
            for i in range(self.n_y_unique):
                partial_results.append(self.hyper_pipes[i].partial_contains(instance))
            best = max(partial_results,key=lambda item:item[0])[1]
            predictions.append(best)

        return predictions

### Benchmark

O HyperPipes consegue ser performático em bases de dados com excessivos atributos e/ou classes bem separadas no espaço amostral, porém geralmente não desempenha bem caso contrário. Quando o algoritmo foi incorporado no pacote de Machine Learning Weka, muitos estudos de benchmark de classificadores para diversos problemas foram realizados.

Delgado e Amorim realizaram um estudo de avaliação envolvendo 179 classificadores oriundos de 17 famílias (discriminant analysis, Bayesian, neural networks, support vector machines, decision trees, rule-based classiﬁers, boosting, bagging, stacking, random forests e outros ensembles, generalized linear models, nearest-neighbors, partial least squares e principal component regression, logistic e multinomial regression, multiple adaptive regression splines e outros métodos), implementados em Weka, R, C e Matlab. Eles usaram 121 bases de dados distintas, representando a maior parte dos presentes na UCI e de outras fontes. Segundo os rankings de Friedman, acurácia média e Cohen k, o HyperPipes ocupou a posição 167 dos 179 classificadores das diversas famílias. Em sua família específica, a de outros métodos, ocupou a posição 8 dentre os 10 presentes.

Em outro estudo de para um problema específico de uso de aprendizado de máquina aplicado a trajetórias dinâmicas de moléculas, HyperPipes obteve o 44 lugar dos 65 classificadores de diversas famílias do Weka, ao passo que foi o primeiro dentre os 3 de categoria outro (ou miscelânea).

Abaixo é feito uma simples demonstração de performance do HiperPipes frente a outros algoritmos.

In [4]:
classifiers_names = ["HyperPipes", "Naive Bayes"]

classifiers = [
    HyperPipes(),
    GaussianNB()
]

datasets_names = ["Iris", "Wine", "Breast Cancer", "Digits"]
datasets = [
    load_iris(),
    load_wine(),
    load_breast_cancer(),
    load_digits()
]

In [5]:
for dataset_name, dataset in zip(datasets_names, datasets):

    X, y = dataset.data, dataset.target

    X = StandardScaler().fit_transform(X)
    
    folds = StratifiedKFold(y, n_folds=10, random_state=8893)
    
    print(dataset_name)
    for name, clf in zip(classifiers_names, classifiers):
        score = cross_val_score(clf, X, y, scoring='accuracy', cv=folds)
        mean = np.mean(score)
        print('* ' + name + ':\t' + str(mean))

Iris
* HyperPipes:	0.9466666666666667
* Naive Bayes:	0.9533333333333334
Wine
* HyperPipes:	0.8953216374269004
* Naive Bayes:	0.9616959064327485
Breast Cancer
* HyperPipes:	0.8153886872353298
* Naive Bayes:	0.9315681444991789
Digits
* HyperPipes:	0.5120985509070034
* Naive Bayes:	0.757390252067086


# Decomposição de Classes

Pelo fato do HyperPipes ser um classificador extremamente simples e estar associado a separação quase que linear das classes, ele acaba performando muito mal em testes de benchmarks com bases de dados de características genéricas. 

Uma maneira de aumentar a performance do algoritmo seria aplicar algum tipo de pré-processamento a entrada dos dados a fim de melhorar os modelos produzidos pelo algoritmo. Estudos propõem a utilização de técnicas de decomposição de classes usando algoritmos de clustering com o objetivo de melhorar a acurácia de classificadores simples – uma vez que a separação de classes prévia torne a nova distribuição mais fácil de ser explorada pelo classificador.

## Implementação Uniforme

Abaixo é implementado um agente de decomposição de classes que recebe como argumento de seu construtor um algoritmo de clusterização qualquer, bem como um parametro *k* do número de clusters por classe. Como pode ser visto na implementação, este decompositor opera de maneira **uniforme** sobre todas as classes, usando o mesmo parâmetro *k* de clusters para todas classes.

In [6]:
class UniformClassDecomposition:
    
    def __init__(self, algorithm, k):
        self.algorithm = algorithm
        self.k = k

    def decompose(self, data_x, data_y):
        self.y_unique_values, self.y_unique_indices = np.unique(data_y, return_inverse=True)
        self.n_y_unique = self.y_unique_values.shape[0]

        renamed_data_x = np.array([])
        renamed_data_y = np.array([])
        for i in range(self.n_y_unique):
            target_class = self.y_unique_values[i]
            target_class_indices = np.where(data_y == target_class)

            data_i_x = data_x[target_class_indices]
            data_i_y = data_y[target_class_indices]
            fitted = self.algorithm.fit(data_i_x)
            labels = fitted.labels_
            
            for j in range(self.k):
                indices_j = np.where(labels == j)
                data_j_x = data_i_x[indices_j]
                data_j_y = data_i_y[indices_j]

                relabed = self.relabel(data_j_y, j)

                renamed_data_x = np.append(renamed_data_x, data_j_x)
                renamed_data_y = np.append(renamed_data_y, relabed)

        X = np.reshape(renamed_data_x, (data_x.shape[0], data_x.shape[1]))
        y = renamed_data_y
        
        return X, y

    def relabel(self, data, cluster_idx):
        return np.array(list(map(lambda item: str(item) + chr(65 + cluster_idx), data)))

### Exemplo

Um exemplo da ação de decomposição é mostrado abaixo.

Originalmente, o dataset de *Wine* possuia 3 classes, com respectivos labels de **0**, **1** e **2**:

In [7]:
dataset = load_wine()
X, y = dataset.data, dataset.target

set(y)

{0, 1, 2}

Após a decomposição de classes uniform, com *k=2*, teremos 6 novas classes.

Cada nova classe será resultado de um cluster, cujo nome é obtido a partir da função *relabel*.
Na implementação acima, o nome original da classe é concatenado com uma caractere, iniciando a indexação em *A*.

In [8]:
k = 2

kmeans = KMeans(n_clusters=k, random_state=42)
cd = UniformClassDecomposition(kmeans, k)

X_cd, y_cd = cd.decompose(X,y)

set(y_cd)

{'0A', '0B', '1A', '1B', '2A', '2B'}

### Benchmark

In [9]:
k = 2

kmeans = KMeans(n_clusters=k, random_state=42)
cd = UniformClassDecomposition(kmeans, k)

for dataset_name, dataset in zip(datasets_names, datasets):

    X, y = dataset.data, dataset.target
    
    X_cd, y_cd = cd.decompose(X,y)

    X_cd = StandardScaler().fit_transform(X_cd)
    
    folds = StratifiedKFold(y_cd, n_folds=10, random_state=8893)
    
    print(dataset_name)
    for name, clf in zip(classifiers_names, classifiers):
        score = cross_val_score(clf, X_cd, y_cd, scoring='accuracy', cv=folds)
        mean = np.mean(score)
        print('* ' + name + ':\t' + str(mean))

Iris
* HyperPipes:	0.8731774761186527
* Naive Bayes:	0.9146606334841628
Wine
* HyperPipes:	0.7833032950022114
* Naive Bayes:	0.8422871517027865
Breast Cancer
* HyperPipes:	0.8001046513393094
* Naive Bayes:	0.8931071865014004
Digits
* HyperPipes:	0.6441753960716742
* Naive Bayes:	0.7207581462390856
